https://www.kaggle.com/competitions/data-science-london-scikit-learn/overview

In [1]:
import pandas as pd

In [2]:
X = pd.read_csv("../Datasets/data-science-london-scikit-learn/train.csv", header=None)
Y = pd.read_csv(
    "../Datasets/data-science-london-scikit-learn/trainLabels.csv", header=None
).values.ravel()
X_test = pd.read_csv(
    "../Datasets/data-science-london-scikit-learn/test.csv", header=None
)
Y = Y[: len(X)]

In [3]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

base_models = [
    (
        "lr",
        make_pipeline(
            StandardScaler(), LogisticRegression(solver="saga", max_iter=1000)
        ),
    ),
    ("rf", RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42)),
    (
        "gb",
        GradientBoostingClassifier(
            n_estimators=200, learning_rate=0.01, max_depth=5, random_state=42
        ),
    ),
]

In [4]:
from sklearn.ensemble import VotingClassifier

vote = VotingClassifier(estimators=base_models, voting="hard")
vote.fit(X, Y)
print("Voting accuracy:", vote.score(X, Y))

Voting accuracy: 0.932


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

preprocessor = ColumnTransformer([("scaler", StandardScaler(), slice(0, 40))])

ensemble_pipeline = Pipeline([("pre", preprocessor), ("model", vote)])

ensemble_pipeline.fit(X, Y)

Y_test = ensemble_pipeline.predict(X_test)

In [21]:
submission = pd.DataFrame(
    {"Id": range(1, len(Y_test) + 1), "Solution": Y_test.astype(int)}
)
submission.to_csv("submission.csv", index=False)
submission.head(10)

,Id,Solution
0,1,1
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,0
7,8,1
8,9,0
9,10,0
